<a href="https://colab.research.google.com/github/ZackPashkin/basic-pipelines-pytorch/blob/main/Pytorch_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!nvidia-smi

Tue Jul 13 09:27:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    32W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/commaai/comma10k.git
!pip install segmentation_models_pytorch

fatal: destination path 'comma10k' already exists and is not an empty directory.


In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader
import cv2
from glob import glob
import os
from tqdm.notebook import tqdm

In [ ]:
len(os.listdir('/content/comma10k/imgs'))
# len(os.listdir('/content/comma10k/imgs2'))

9888

In [ ]:
len(os.listdir('/content/comma10k/masks'))
# len(os.listdir('/content/comma10k/masks2'))

9888

In [ ]:
imgs_path = "/content/comma10k/imgs"
masks_path = "/content/comma10k/masks"
imgs_path_val = "/content/comma10k/imgs2"
masks_path_val = "/content/comma10k/masks2"

image_size = 256


device = "cuda" if torch.cuda.is_available() else "cpu"


batch_size = 16
epochs = 10
num_workers = 2

# Dataset

In [ ]:
class SegmentationDataset:
  def __init__(self, imgs_path, masks_path, transform=None):
    self.imgs_path = sorted(glob(f"{imgs_path}/*.png"))#[:32]
    self.masks_path = sorted(glob(f"{masks_path}/*.png"))#[:32]
    self.transform = transform

  def __len__(self):
    assert len(self.imgs_path) == len(self.masks_path)
    return len(self.imgs_path)

  def __getitem__(self, idx):
    img = cv2.imread(self.imgs_path[idx])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # read in grayscale add 0
    mask = cv2.imread(self.masks_path[idx], 0)
    if self.transform:
      norm_img = transforms.Normalize((0.485,0.456,0.406),
                                  (0.229, 0.224, 0.225))
      img = norm_img(self.transform(img))
      # add channel to beginning
      mask = self.transform(mask).squeeze(0)
      # for this dataset
      # each pixel represents a class
      mask[mask < 0.255] = 4.0
      mask[mask < 0.35] = 3.0
      mask[mask < 0.443] = 2.0
      mask[mask < 0.54] = 1.0
      mask[mask < 0.7] = 0.0
    return img, mask

In [ ]:
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Resize((image_size,image_size))
])

In [ ]:
train_dataset = SegmentationDataset(imgs_path,masks_path,transform)
train_loader = DataLoader(train_dataset, batch_size, num_workers=num_workers, shuffle=True)

In [ ]:
val_dataset = SegmentationDataset(imgs_path_val,masks_path_val,transform)
val_loader = DataLoader(val_dataset, batch_size, num_workers=num_workers, shuffle=True)

In [ ]:
# test val_loader
for data in val_loader:
  print(data)
  break

[tensor([[[[-2.0969, -2.0816, -2.0909,  ..., -2.0590, -2.0518, -2.0053],
          [-2.0810, -2.0973, -2.0837,  ..., -2.0323, -2.0552, -2.0620],
          [-2.0703, -2.0997, -2.0682,  ..., -2.0443, -2.0494, -2.0939],
          ...,
          [-2.1008, -2.0837, -2.0991,  ..., -2.0665, -2.0665, -2.0665],
          [-2.1008, -2.0837, -2.0837,  ..., -2.0665, -2.0665, -2.0962],
          [-2.1008, -2.0837, -2.1008,  ..., -2.0738, -2.0665, -2.1008]],

         [[-2.0189, -2.0157, -2.0256,  ..., -1.9824, -1.9857, -2.0182],
          [-2.0027, -2.0357, -2.0182,  ..., -2.0007, -2.0066, -2.0229],
          [-2.0045, -2.0347, -2.0024,  ..., -1.9884, -2.0007, -1.9794],
          ...,
          [-2.0182, -2.0357, -2.0165,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0182, -2.0357, -2.0007,  ..., -2.0357, -2.0182, -2.0229],
          [-2.0182, -2.0357, -2.0182,  ..., -2.0357, -2.0357, -2.0357]],

         [[-1.7645, -1.6978, -1.7372,  ..., -1.6667, -1.7023, -1.6724],
          [-1.7577, -1.6789, 

In [ ]:
# test train_loader
for data in train_loader:
  print(data)
  break

[tensor([[[[ 0.1456,  0.1878,  0.0605,  ...,  0.3138,  0.4095,  0.3212],
          [ 0.1216,  0.1737,  0.1573,  ...,  0.3438,  0.4132,  0.3481],
          [ 0.1177,  0.1480,  0.1426,  ...,  0.4508,  0.4275,  0.3652],
          ...,
          [-1.5269, -1.5185, -1.5699,  ..., -1.4500, -1.4500, -1.4500],
          [-1.5263, -1.5185, -1.5699,  ..., -1.4500, -1.4500, -1.4500],
          [-1.5325, -1.5457, -1.5563,  ..., -1.4500, -1.4500, -1.4500]],

         [[ 1.2237,  1.1968,  1.2170,  ...,  1.4657,  1.4482,  1.4069],
          [ 1.1991,  1.1824,  1.2357,  ...,  1.4963,  1.4447,  1.4482],
          [ 1.2477,  1.2787,  1.2731,  ...,  1.4832,  1.4594,  1.4307],
          ...,
          [-1.1570, -1.1604, -1.1429,  ..., -1.0903, -1.0903, -1.0903],
          [-1.1564, -1.1604, -1.1429,  ..., -1.0903, -1.0903, -1.0903],
          [-1.1779, -1.1604, -1.1567,  ..., -1.0903, -1.0903, -1.0903]],

         [[ 2.3643,  2.4246,  2.3960,  ...,  2.5354,  2.5529,  2.5358],
          [ 2.3398,  2.4103, 

# Model

In [ ]:
class Unet(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = smp.Unet(in_channels=3, classes=5)

  def forward(self, x):
    x = self.model(x)
    return x

# Train

In [ ]:
# model simple test
model = Unet()
model(torch.rand(4,3,224,224))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


tensor([[[[ 5.7292e-01,  2.2304e-01,  6.3649e-01,  ...,  4.7336e-01,
            2.8115e-01,  1.4010e-01],
          [ 4.6125e-01,  1.3378e+00,  1.3373e+00,  ...,  3.2854e-01,
            1.2735e+00,  7.7808e-01],
          [ 6.8004e-02,  8.1855e-01,  1.5387e+00,  ...,  3.8551e-01,
            1.1491e+00,  1.1199e+00],
          ...,
          [ 1.0236e+00, -6.1182e-02, -5.6831e-02,  ..., -2.2089e-01,
            2.1569e-01,  7.8703e-01],
          [ 9.2489e-01,  6.6253e-01,  1.3307e+00,  ...,  1.0988e+00,
            1.3010e+00,  1.3677e+00],
          [ 1.7659e-01, -2.8635e-02,  7.3312e-01,  ...,  1.3885e+00,
            1.6110e+00,  9.0388e-01]],

         [[ 4.0085e-01,  2.7293e-01, -9.9805e-03,  ...,  4.2785e-01,
            6.2467e-01, -3.5848e-01],
          [ 1.6714e-01,  6.3837e-01, -1.9049e-01,  ..., -1.7091e-01,
           -1.1775e-01, -2.5377e-02],
          [ 1.2200e-01,  6.9641e-01, -5.3165e-01,  ..., -7.8964e-02,
           -1.9533e-01,  2.4231e-02],
          ...,
     

In [ ]:
model = Unet().to(device)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
lr = 0.001
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

In [ ]:
plt.ion()
import matplotlib.pyplot as plt
from random import randint
for epoch in tqdm(range(epochs),"Epoch"):
  epoch_loss_train = 0
  best_val_loss = 0

  tqdm_loop = tqdm(enumerate(train_loader),'Steps',total=len(train_loader))
  for i,(img_batch,mask_batch) in tqdm_loop:
    img_batch = img_batch.to(device, dtype=torch.float32)
    mask_batch = mask_batch.to(device, dtype=torch.long)


    prediction_mask = model(img_batch)
    loss = criterion(prediction_mask, mask_batch)

    loss.backward()

    optimizer.step()

    epoch_loss_train += loss.item()
    tqdm_loop.set_postfix(loss=loss.item())

    epoch_loss = epoch_loss_train / len(train_loader)
  
  print(f'Train loss: {epoch_loss:9.4f}')


  with torch.no_grad():
    epoch_loss_val = 0
    model.eval()
    for i,(img_batch,mask_batch) in enumerate(val_loader):
      img_batch = img_batch.to(device, dtype=torch.float32)
      mask_batch = mask_batch.to(device, dtype=torch.long)

      prediction_mask = model(img_batch)
      loss = criterion(prediction_mask, mask_batch)

      epoch_loss_val += loss.item()
      epoch_loss_val = epoch_loss_val / len(val_loader)

      if i  == randint(0,len(val_loader)):
    
        mask = prediction_mask.argmax(1)[0].cpu().detach().numpy()
        print(mask)
        # plt.imshow(mask*255, cmap='gray')


     
      

    if epoch_loss_val < best_val_loss:
      best_val_loss = epoch_loss
      model.save()
      print("Sheckpoints are saved")
  print(f'Val loss: {epoch_loss_val:9.4f}')


Train loss:    1.2574
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]
Val loss:    1.0823



Train loss:       nan
Val loss:       nan



Train loss:       nan
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Val loss:       nan



Train loss:       nan
Val loss:       nan



Train loss:       nan
Val loss:       nan



Train loss:       nan
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Val loss:       nan



Train loss:       nan
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Val loss:       nan



Train loss:       nan
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Val loss:       nan



Train loss:       nan
Val loss:       nan



Train loss:       nan
Val loss:       nan



# Note

```transforms.Normalize((0.485,0.456,0.406),(0229, 0.224.0.225)) ```

``` 
      mask[mask < 0.255] = 4.0
      mask[mask < 0.35] = 3.0
      mask[mask < 0.443] = 2.0
      mask[mask < 0.54] = 1.0
      mask[mask < 0.7] = 0.0
```                                